In [1]:
!which pip && which python

/home/buka2004/PTQ-LLM-MIPT/.venv/bin/pip
/home/buka2004/PTQ-LLM-MIPT/.venv/bin/python


# Imports

In [ ]:
import time
import argparse
import logging
import math
import os
import random
from pathlib import Path
from re import L

import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)
# from transformers.file_utils import get_full_repo_name
from transformers.utils.versions import require_version

import deepspeed
from deepspeed.compression.compress import init_compression, redundancy_clean
from deepspeed.compression.helper import recursive_getattr
from deepspeed.compression.helper import convert_conv1d_to_linear

import numpy as np
from transformers.modeling_utils import Conv1D

from torch.utils.tensorboard import SummaryWriter

from smoothquant.smoothquant.calibration import get_act_scales
from smoothquant.smoothquant.smooth import smooth_lm

# Set python path

In [ ]:
! export PYTHONPATH="/home/buka2004/PTQ-LLM-MIPT:$PYTHONPATH"

# Set params

In [ ]:
CONFIG='/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/config/ds_config_W8A8_Qgroup64_fp32.json'
SAVE_PATH='/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/out/ZeroQuant/W8A8_quantization_lkd_saving'
os.makedirs(SAVE_PATH, exist_ok=True)

# Initialize model

In [ ]:
MODEL='openai-community/gpt2-large'
config = AutoConfig.from_pretrained(CONFIG)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    from_tf=bool(".ckpt" in MODEL),
)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 69a6e7d5-ca5d-4cc5-ae7d-0d91714b665c)')' thrown while requesting HEAD https://huggingface.co/openai-community/gpt2-large/resolve/main/config.json
Retrying in 1s [Retry 1/5].


In [ ]:
# Configuration
MODEL = 'openai-community/gpt2-large'
CONFIG = '/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/config/ds_config_W8A8_Qgroup64_fp32.json'
SAVE_PATH = '/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/out/ZeroQuant/W8A8_quantization_lkd_saving'

import os
import torch
import deepspeed
from transformers import AutoModelForCausalLM

# Create save directory
os.makedirs(SAVE_PATH, exist_ok=True)

# Load model
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     from_tf=bool(".ckpt" in MODEL),
# )

# Initialize DeepSpeed with quantization config and required training parameters
ds_config = {
    "train_batch_size": 1,  # Required by DeepSpeed
    "train_micro_batch_size_per_gpu": 1,  # Required by DeepSpeed
    "gradient_accumulation_steps": 1,
    "steps_per_print": 1,
    "fp16": {
        "enabled": False
    },
    "bf16": {
        "enabled": False
    },
    "zero_optimization": {
        "stage": 0,  # Disable ZeRO for quantization
    },
    "quantization": {
        "enabled": True,
        "quantize_weight_in_forward": False,
        "quantize_groups": 64,
        "wq8": True,
        "aq8": True,
    }
}

model = AutoModelForCausalLM.from_pretrained(MODEL, torch_dtype=torch.float16)
ds_engine, _, _, _ = deepspeed.initialize(model=model, config_params=ds_config)
lean_state_dict = deepspeed.checkpoint.utils.clone_tensors_for_torch_save(ds_engine.module.state_dict())
ds_engine.module.save_pretrained("lean_after", state_dict=lean_state_dict)

# # Initialize DeepSpeed model
# model_engine, _, _, _ = deepspeed.initialize(
#     model=model,
#     config=ds_config
# )

# model_engine.save_checkpoint(SAVE_PATH)

print(f"Quantized model saved to: {SAVE_PATH}")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ab20088b-53ec-422c-b554-37f5b57b90b3)')' thrown while requesting HEAD https://huggingface.co/openai-community/gpt2-large/resolve/main/config.json
Retrying in 1s [Retry 1/5].


Quantized model saved to: /home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/out/ZeroQuant/W8A8_quantization_lkd_saving


In [ ]:
"""
Alternative: Save quantized model by creating a modified config
that stores quantized weights in the state dict.
"""
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
from transformers.modeling_utils import Conv1D
from deepspeed.compression.compress import init_compression, redundancy_clean
from deepspeed.compression.helper import convert_conv1d_to_linear

# Configuration
MODEL = 'openai-community/gpt2-large'
CONFIG_PATH = '/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/config/ds_config_W8A8_Qgroup64_fp32.json'
SAVE_PATH = '/home/buka2004/PTQ-LLM-MIPT/DeepSpeedExamples/compression/gpt2/out/ZeroQuant/W8A8_quantization_lkd_saving'

os.makedirs(SAVE_PATH, exist_ok=True)

# Load and modify config to disable runtime quantization
with open(CONFIG_PATH, 'r') as f:
    ds_config = json.load(f)

# Set quantize_weight_in_forward to False to store quantized weights
ds_config['compression_training']['weight_quantization']['shared_parameters']['quantize_weight_in_forward'] = False

# Save modified config
modified_config_path = os.path.join(SAVE_PATH, 'ds_config_modified.json')
with open(modified_config_path, 'w') as f:
    json.dump(ds_config, f, indent=2)

print("Loading model...")
model = AutoModelForCausalLM.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

print("Converting Conv1D to Linear...")
model = convert_conv1d_to_linear(model, Conv1D)

print("Applying compression with modified config...")
model = init_compression(model, modified_config_path)

print("Cleaning redundant wrappers...")
model = redundancy_clean(model, modified_config_path)

print("\nSaving quantized model...")
# Save the model
model_to_save = model.module if hasattr(model, 'module') else model
state_dict = model_to_save.state_dict()

# Save state dict
torch.save(state_dict, os.path.join(SAVE_PATH, 'pytorch_model.bin'))

# Save config and tokenizer
model_to_save.config.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

# Report size
checkpoint_size = os.path.getsize(os.path.join(SAVE_PATH, 'pytorch_model.bin'))
checkpoint_size_mb = checkpoint_size / (1024 * 1024)
checkpoint_size_gb = checkpoint_size / (1024 * 1024 * 1024)

print(f"\n{'='*60}")
print(f"Model saved to: {SAVE_PATH}")
print(f"Checkpoint size: {checkpoint_size:,} bytes ({checkpoint_size_mb:.2f} MB / {checkpoint_size_gb:.2f} GB)")
print(f"\nTo load:")
print(f"  model = AutoModelForCausalLM.from_pretrained('{SAVE_PATH}')")

Loading model...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: b455e7de-cbb4-4e01-aeff-4ff104ac5ca1)')' thrown while requesting HEAD https://huggingface.co/openai-community/gpt2-large/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Exception ignored in: <function DeepSpeedEngine.__del__ at 0x7f70a476f040>
Traceback (most recent call last):
  File "/home/buka2004/PTQ-LLM-MIPT/.venv/lib/python3.8/site-packages/deepspeed/runtime/engine.py", line 527, in __del__
    def __del__(self):
KeyboardInterrupt: 


Converting Conv1D to Linear...


KeyboardInterrupt: 